In [21]:
from lyricsgenius import Genius
import pandas as pd

In [22]:
f = open('../token.secret', 'r')
token = f.read()
f.close()

In [23]:
genius = Genius(token)

First, we look for all songs

In [ ]:
artist = genius.search_artist('Drake', max_songs=3)

In [ ]:
artist.id

In [ ]:
songs = []
page = 1
while page:
    print(page)
    request = genius.artist_songs(artist_id = 130, page=page, per_page=50)
    songs.extend(request['songs'])
    page = request['next_page']

In [ ]:
len(songs)

In [104]:
songs

[{'annotation_count': 0,
  'api_path': '/songs/4317017',
  'artist_names': 'Drake',
  'full_title': '. by\xa0Drake',
  'header_image_thumbnail_url': 'https://images.genius.com/7dbc2190892ad991a1abdd33c65990b6.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/7dbc2190892ad991a1abdd33c65990b6.623x623x1.jpg',
  'id': 4317017,
  'lyrics_owner_id': 7295142,
  'lyrics_state': 'complete',
  'path': '/Drake--annotated',
  'pyongs_count': None,
  'relationships_index_url': 'https://genius.com/Drake--sample',
  'release_date_components': {'year': 2019, 'month': 2, 'day': 15},
  'release_date_for_display': 'February 15, 2019',
  'release_date_with_abbreviated_month_for_display': 'Feb. 15, 2019',
  'song_art_image_thumbnail_url': 'https://images.genius.com/7dbc2190892ad991a1abdd33c65990b6.300x300x1.jpg',
  'song_art_image_url': 'https://images.genius.com/7dbc2190892ad991a1abdd33c65990b6.623x623x1.jpg',
  'stats': {'unreviewed_annotations': 0, 'hot': False},
  'title': '.',
  'title_

Many of the returned songs are not actually songs (for instance, apology for autism lyric).

Thus, we'll just focus on songs from solo studio albums.

This will exclude Her Loss and What a Time to Be Alive.

In [4]:
albums = genius.artist_albums(artist_id = 130)

In [7]:
album_df_array = []
for album in albums.get('albums'):
    album_id = album.get('id')
    album_name = album.get('name')
    release_date = album.get('release_date_components')
    album_tracks = genius.album_tracks(album_id = album_id)
    for item in album_tracks.get('tracks'):
        song_id = item.get('song').get('id')
        song_name = item.get('song').get('full_title')
        song_name = song_name.split(' by\xa0')[0]
        album_df_array.append(pd.DataFrame({'album_name':[album_name], 
                                            'album_id':[album_id], 
                                            'song_id':[song_id],
                                            'song_name':[song_name],
                                           'release_date':[release_date.get('year')]}))

In [8]:
album_df = pd.concat(album_df_array)#['album_id'].value_counts()

In [9]:
prohib_list = ['Scary Hours 3*', 'FOR ALL THE DOGS']

In [10]:
album_df = album_df[~album_df['album_name'].isin(prohib_list)].copy()

In [11]:
album_df['album_name'].value_counts()

Comeback Season                         25
Scorpion                                25
More Life                               23
Take Care                               22
Room for Improvement                    22
Certified Lover Boy                     21
Views                                   20
If You’re Reading This It’s Too Late    19
So Far Gone                             18
Care Package                            17
Thank Me Later                          16
Nothing Was the Same                    16
Dark Lane Demo Tapes                    14
Honestly, Nevermind                     14
Young Sweet Jones                       12
So Far Gone (EP)                         7
Drake Demo Disk                          4
Scary Hours 2                            3
Scary Hours                              2
The Best in the World Pack               2
Name: album_name, dtype: int64

In [35]:
album_type_dict = {'Comeback Season':'Compilation album', 
 'Scorpion':'Studio album', 
 'More Life':'Playlist', 
 'Take Care':'Studio album', 
 'Room for Improvement':'Mixtape', 
 'Certified Lover Boy':'Studio album', 
 'Views':'Studio album', 
 'If You’re Reading This It’s Too Late':'Mixtape', 
 'So Far Gone':'EP', 
 'Care Package':'Compilation Album', 
 'Nothing Was the Same':'Studio album', 
 'Thank Me Later':'Studio album', 
 'Honestly, Nevermind':'Studio album', 
 'Dark Lane Demo Tapes':'Mixtape', 
 'Young Sweet Jones':'Mixtape', 
 'So Far Gone (EP)':'EP', 
 'Drake Demo Disk':'EP', 
 'Scary Hours':'EP', 
 'Scary Hours 2':'EP', 
 'The Best in the World Pack':'EP'}

In [36]:
album_df['album_type'] = album_df['album_name'].apply(lambda x: album_type_dict.get(x))

In [14]:
album_df

,album_name,album_id,song_id,song_name,release_date,album_type
0,"Honestly, Nevermind",880669,8111078,Intro,2022,Studio album
0,"Honestly, Nevermind",880669,8110883,Falling Back,2022,Studio album
0,"Honestly, Nevermind",880669,8110884,Texts Go Green,2022,Studio album
0,"Honestly, Nevermind",880669,8110885,Currents,2022,Studio album
0,"Honestly, Nevermind",880669,8110886,A Keeper,2022,Studio album
...,...,...,...,...,...,...
0,Room for Improvement,2625,3033405,U.P.A. (Outro),2006,Mixtape
0,Drake Demo Disk,516437,3351,City Is Mine,2006,EP
0,Drake Demo Disk,516437,4427019,Inst,2006,EP
0,Drake Demo Disk,516437,4427020,Acapella,2006,EP


Retrieve lyrics for each song

In [25]:
def clean_lyrics(lyrics):
    clean_lyrics = ''.join(lyrics.split('Lyrics')[1:])
    replace_dict = {'\n':' ',
                    "\'":'', 
                    '’':'',
                    '[Outro]':' ',
                    '\u205f':' ',
                    'See Drake LiveGet tickets as low as $48':' ',
                    '[Intro]':' ', 
                    '[Break]':' ', 
                    '[Chorus]':' ',
                    '[Bridge]':' ',
                    '[Coda]':' ',
                    '[Verse]':' ',
                    '4Embed':' ',
                    '8Embed':' ',
                    'You might also like': ' '}
    for num in range(0,20):
        replace_dict[str(num)+'Embed'] = ' '
    for key, value in replace_dict.items():
        clean_lyrics = clean_lyrics.replace(key, value)
    return clean_lyrics

In [26]:
%%time
lyrics_array = []
for song_id in album_df['song_id'].values:
    for _ in range(100):
        try:
            print(song_id)
            lyrics = genius.lyrics(song_id=song_id, remove_section_headers=True)
            break
        except:
            pass
        
    if not lyrics:
        lyrics_array.append('')
    else:
        lyrics = clean_lyrics(lyrics)
        lyrics_array.append(lyrics)

8111078
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Drake-intro-lyrics
8110883
8110884
8110885
8110886
8110887
8110888
8110892
8110894
8110895
8110896
8110898
8110893
8110900
6859296
7165569
7165581
7165576
5367025
6833653
7139486
5312186
7163722
7165573
7165570
7165568
7165580
7165567
5851754
7165583
7165564
7165563
7165582
7165549
7165548
6515213
6555267
6555268
5433444
5127477
5310327
5520781
5402847
4825634
5433461
5433550
5520765
4613586
5520818
5447963
5235956
5113565
50202
176772
546639
51360
384908
393612
2489756
124418
1497
748776
176774
55579
55589
240199
140797
776
1865293
4629363
4629364
3807729
3807730
3797175
3807731
3315890
3745302
3807743
3807746
3807748
3807749
3807753
3631865
3807755
3807747
3807756
3580132
3807760
3807763
3807765
3807754
3807759
3807759
3807766
3807744
3807764
3807750
3315890
3248485
2992433
2992774
3028662
3028667
3006986
3028673
3028677
3028680
3028682
3028685
3028687
3028693
3028696
302

In [27]:
album_df['lyrics'] = lyrics_array

In [39]:
album_df.to_csv('../data/supplementary_data_1.csv', index=False)